# Transforming data from silver and writing to gold 

In [0]:
dbutils.fs.ls("mnt/silver/public")

[FileInfo(path='dbfs:/mnt/silver/public/customers/', name='customers/', size=0, modificationTime=1718021370000),
 FileInfo(path='dbfs:/mnt/silver/public/orders/', name='orders/', size=0, modificationTime=1718021373000),
 FileInfo(path='dbfs:/mnt/silver/public/products/', name='products/', size=0, modificationTime=1718021376000)]

In [0]:
table_names = [table.name.split('/')[0] for table in dbutils.fs.ls("mnt/silver/public/")]

print(table_names)


['customers', 'orders', 'products']


In [0]:
for table in table_names:
    input_path = f"/mnt/silver/public/{table}/"
    df = spark.read.format('delta').load(input_path)
    if table == "orders":
        from pyspark.sql.functions import split, col

        split_col = split(df['order_date'], '/')

        df = df.withColumn('month', split_col.getItem(0)) \
            .withColumn('day', split_col.getItem(1)) \
            .withColumn('year', split_col.getItem(2))

    output_path = f"/mnt/gold/public/{table}/"
    df.write.format('delta').mode("overwrite").save(output_path)